In [7]:
import data_structure as ds
from data_structure import Gender
import pandas as pd
import numpy as np

In [8]:
df = ds.search_all()

# df = df[(df['round_'] == 5) | (df['round_'] == 6)]
# df = df[(df['round_'] == 7) | (df['round_'] == 5) | (df['round_'] == 6)]
df = df[df['round_'] == 7]

In [9]:
male_mask = df['gender'] == Gender.Male
df_male = df[male_mask]
df_female = df[df['gender'] == Gender.Female]
df_neutral = df[df['gender'] == Gender.Neutral]

In [10]:
deserved_group_df = pd.concat(
                    [df_male.value_counts('deserved_salary').sort_index(),
                      df_female.value_counts('deserved_salary').sort_index(),
                      df_neutral.value_counts('deserved_salary').sort_index()],
                        axis=1).reset_index()
deserved_group_df.replace(np.nan, 0, inplace=True)
deserved_group_df = deserved_group_df.apply(lambda x: x.astype(int), axis=1)
deserved_group_df.columns = ['salary',	'male', 'female', 'neutral']

current_group_df = pd.concat(
                    [df_male.value_counts('current_salary').sort_index(),
                     df_female.value_counts('current_salary').sort_index(),
                     df_neutral.value_counts('current_salary').sort_index()],
                        axis=1).reset_index()
current_group_df.replace(np.nan, 0, inplace=True)
current_group_df = current_group_df.apply(lambda x: x.astype(int), axis=1)
current_group_df.columns = ['salary',	'male', 'female', 'neutral']

deserved_group_df = deserved_group_df.sort_values('salary')
current_group_df = current_group_df.sort_values('salary')

display(current_group_df)
display(deserved_group_df)

deserved_group_df['salary_group'] = pd.cut(deserved_group_df['salary'],
                                            [0,80, 89, 150],
                                            labels=['<=80', '80-89', '>89'])
current_group_df['salary_group'] = pd.cut(current_group_df['salary'],
                                            [0, 60, 70, 100],
                                            labels=['>60', '60-70', '>70'])

deserved_group_df = deserved_group_df.groupby('salary_group').sum().reset_index().drop(columns='salary')
current_group_df = current_group_df.groupby('salary_group').sum().reset_index().drop(columns='salary')

print('des')
display(deserved_group_df)

print('cur')
display(current_group_df)

,salary,male,female,neutral
6,50,0,1,0
0,55,1,5,1
1,60,222,340,227
2,65,38,38,35
3,70,228,112,219
4,75,8,3,16
5,80,3,1,2


,salary,male,female,neutral
7,65,0,1,0
0,70,1,5,0
1,75,199,302,190
2,80,52,73,70
3,85,161,87,149
4,90,83,31,87
5,95,2,1,2
6,100,2,0,2


des


C:\Users\pvest\AppData\Local\Temp\ipykernel_12520\931211682.py:32: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  deserved_group_df = deserved_group_df.groupby('salary_group').sum().reset_index().drop(columns='salary')
C:\Users\pvest\AppData\Local\Temp\ipykernel_12520\931211682.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  current_group_df = current_group_df.groupby('salary_group').sum().reset_index().drop(columns='salary')


,salary_group,male,female,neutral
0,<=80,252,381,260
1,80-89,161,87,149
2,>89,87,32,91


cur


,salary_group,male,female,neutral
0,>60,223,346,228
1,60-70,266,150,254
2,>70,11,4,18


In [11]:
from scipy import stats

observed_deserved = deserved_group_df[['male', 'female', 'neutral']].to_numpy()

mf_obs = [observed_deserved[:,0], observed_deserved[:,1]], 'male-female'
mn_obs = [observed_deserved[:,0], observed_deserved[:,2]], 'male-neutral'
fn_obs = [observed_deserved[:,1], observed_deserved[:,2]], 'female-neutral'

for obs, descr in [mf_obs, mn_obs, fn_obs]:
    chi2_stat, p_val, dof, expected = stats.chi2_contingency(obs)
    print(f"Chi-square Test {descr}:\nChi2 Stat: {chi2_stat}, P-value: {p_val}, Degrees of Freedom: {dof}")
print('\n\n')

observed_current = current_group_df[['male', 'female', 'neutral']].to_numpy()
mf_des = [observed_current[:,0], observed_deserved[:,1]], 'male-female'
mn_des = [observed_current[:,0], observed_deserved[:,2]], 'male-neutral'
fn_des = [observed_current[:,1], observed_deserved[:,2]], 'female-neutral'

for obs, descr in [mf_des, mn_des, fn_des]:
    chi2_stat, p_val, dof, expected = stats.chi2_contingency(obs)
    print(f"Chi-square Test {descr}:\nChi2 Stat: {chi2_stat}, P-value: {p_val}, Degrees of Freedom: {dof}")


Chi-square Test male-female:
Chi2 Stat: 73.78991275458355, P-value: 9.478159990574176e-17, Degrees of Freedom: 2
Chi-square Test male-neutral:
Chi2 Stat: 0.6794037694816963, P-value: 0.7119825439884577, Degrees of Freedom: 2
Chi-square Test female-neutral:
Chi2 Stat: 67.42982223629582, P-value: 2.2792929307160146e-15, Degrees of Freedom: 2



Chi-square Test male-female:
Chi2 Stat: 142.3546451637392, P-value: 1.2248462194159119e-31, Degrees of Freedom: 2
Chi-square Test male-neutral:
Chi2 Stat: 98.56500873791109, P-value: 3.9525766271171066e-22, Degrees of Freedom: 2
Chi-square Test female-neutral:
Chi2 Stat: 91.88164915417788, P-value: 1.1172577916845978e-20, Degrees of Freedom: 2
